In [25]:
# import libraries

import requests
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import re

In [26]:
# scrape site

url = "https://www.propertyguru.com.sg/property-for-sale?market=residential&property_type=N&property_type_code[]=CONDO&property_type_code[]=APT&district_code[]=D09&district_code[]=D10&district_code[]=D11"

scraper = cloudscraper.create_scraper()
req = scraper.get(url)
soup = BeautifulSoup(req.text,"html.parser")


In [27]:
# Get Area

info_list_1=soup.find_all("li",{"class","listing-floorarea pull-left"})

area_all=[]

for i in range(0,49,2):
    area = info_list_1[i].text
    area_num=float(re.sub("sqft","",area))
    area_all.append(area_num)

len(area_all)

25

In [28]:
# Get unit price

psf_all=[]

for i in range(1,50,2):
    unitPrice = info_list_1[i].text
    UP_li=re.split(r"\s+",unitPrice)
    unitPrice_num=float(UP_li[1].replace(",",""))
    psf_all.append(unitPrice_num)
    
len(psf_all)

25

In [29]:
# Get Project Name

names=[]
names_all=[]

name_list=soup.find_all("a",{"class","nav-link"})

for a in name_list:
    name=a.text
    names.append(name)
    
for i in range(0,75,3):
    names_all.append(names[i])
    
len(names_all)

25

In [30]:
# Get Links

all_links=[]
links=[]

link_list=soup.find_all("a",{"class","nav-link"})

for b in link_list:
    all_links.append(b.get("href"))

for i in range(0,75,3):
    link=all_links[i]
    links.append(link)
    
len(links)

25

In [31]:
# Get distance to mrt

dist_list=soup.find_all("ul",{"class","listing-features pull-left"})

dists=[]
dists2=[]

for c in dist_list:
    dist=re.findall(r'\d+ m',c.text)
    dists.append(dist)

for i in range (1,50,2):
    d=dists[i][1]
    d=float(d.replace("m",""))
    dists2.append(d)

len(dists2)

25

In [32]:
# Get Features

features=[]
feat_list=soup.find_all("ul",{"class","listing-property-type"})

for ea in feat_list:
    result=ea.text.split("\n")
    features.append(result)

features=features[:25]
features

[['', 'New Project: 2023', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2025', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2026', 'Apartment', 'Freehold', ''],
 ['', 'Apartment', 'Freehold', 'Built: 2005', ''],
 ['', 'Condominium', 'Freehold', 'Built: 2013', ''],
 ['', 'Condominium', 'Freehold', 'Built: 2003', ''],
 ['', 'New Project: 2022', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2022', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2022', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2023', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2023', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2023', 'Condominium', 'Freehold', ''],
 ['', 'New Project: 2023', 'Condominium', 'Freehold', ''],
 ['', 'Condominium', 'Freehold', 'Built: 2012', ''],
 ['', 'Condominium', 'Freehold', 'Built: 2017', ''],
 ['', 'Condominium', 'Freehold', 'Built: 2011', ''],
 ['', 'Condominium', '99-year Leasehold', 'Built: 2015', ''],
 ['', 'Condominium', 'Freehold', '

In [33]:
# Create DataFrame


all=pd.DataFrame({"Name":names_all,"Links":links,"Area":area_all,"Unit_Price":psf_all,"Dist_to_mrt":dists2,"Features":features})
all.head()

,Name,Links,Area,Unit_Price,Dist_to_mrt,Features
0,Pullman Residences,https://www.propertyguru.com.sg/listing/233376...,1163.0,2784.18,80.0,"[, New Project: 2023, Condominium, Freehold, ]"
1,Hyll on Holland,https://www.propertyguru.com.sg/listing/236369...,614.0,2752.44,880.0,"[, New Project: 2025, Condominium, Freehold, ]"
2,enchante,https://www.propertyguru.com.sg/listing/242088...,1281.0,2701.01,340.0,"[, New Project: 2026, Apartment, Freehold, ]"
3,Proximo,https://www.propertyguru.com.sg/listing/243760...,1119.0,2412.87,450.0,"[, Apartment, Freehold, Built: 2005, ]"
4,The Ritz-Carlton Residences,https://www.propertyguru.com.sg/listing/242466...,6501.0,6921.99,610.0,"[, Condominium, Freehold, Built: 2013, ]"


In [34]:
# Create PptyTpe and Tenure columns

PptyType=[]
Tenure=[]

for row in all["Features"]:
    if "Condominium" in row:
        PptyType.append(2)
    else:
        PptyType.append(1)
    if "Freehold" in row:
        Tenure.append(2)
    else:
        Tenure.append(1)

all["PptyType"]=PptyType
all["Tenure"]=Tenure

all.head()

,Name,Links,Area,Unit_Price,Dist_to_mrt,Features,PptyType,Tenure
0,Pullman Residences,https://www.propertyguru.com.sg/listing/233376...,1163.0,2784.18,80.0,"[, New Project: 2023, Condominium, Freehold, ]",2,2
1,Hyll on Holland,https://www.propertyguru.com.sg/listing/236369...,614.0,2752.44,880.0,"[, New Project: 2025, Condominium, Freehold, ]",2,2
2,enchante,https://www.propertyguru.com.sg/listing/242088...,1281.0,2701.01,340.0,"[, New Project: 2026, Apartment, Freehold, ]",1,2
3,Proximo,https://www.propertyguru.com.sg/listing/243760...,1119.0,2412.87,450.0,"[, Apartment, Freehold, Built: 2005, ]",1,2
4,The Ritz-Carlton Residences,https://www.propertyguru.com.sg/listing/242466...,6501.0,6921.99,610.0,"[, Condominium, Freehold, Built: 2013, ]",2,2


In [35]:
# Calculate Valuation

all["Valuation"]=(all["Area"]*0.2964)-(all["PptyType"]*89.3533)+(all["Tenure"]*384.7917)-(all["Dist_to_mrt"]*0.0396)+2061.99
all["over_under_valuation"]=(all["Valuation"]-all["Unit_Price"])/all["Valuation"]*100

all.head()

,Name,Links,Area,Unit_Price,Dist_to_mrt,Features,PptyType,Tenure,Valuation,over_under_valuation
0,Pullman Residences,https://www.propertyguru.com.sg/listing/233376...,1163.0,2784.18,80.0,"[, New Project: 2023, Condominium, Freehold, ]",2,2,2994.4120,7.020811
1,Hyll on Holland,https://www.propertyguru.com.sg/listing/236369...,614.0,2752.44,880.0,"[, New Project: 2025, Condominium, Freehold, ]",2,2,2800.0084,1.698866
2,enchante,https://www.propertyguru.com.sg/listing/242088...,1281.0,2701.01,340.0,"[, New Project: 2026, Apartment, Freehold, ]",1,2,3108.4445,13.107344
3,Proximo,https://www.propertyguru.com.sg/listing/243760...,1119.0,2412.87,450.0,"[, Apartment, Freehold, Built: 2005, ]",1,2,3056.0717,21.046682
4,The Ritz-Carlton Residences,https://www.propertyguru.com.sg/listing/242466...,6501.0,6921.99,610.0,"[, Condominium, Freehold, Built: 2013, ]",2,2,4555.6072,-51.944399


In [36]:
# Output results

all=all.sort_values(by="over_under_valuation",ascending=False)
all.to_csv("output.csv")